<a href="https://colab.research.google.com/github/HuyenTrang457/MCM/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler


Clean Data:

1. Xóa bỏ cột serve_width, serve_depth,  return_depth vì chúng tôi dường như không làm việc với chúng.
2. Nhận thấy chỉ có speed_mph bị lẫn các giá trị NA vậy nên ta phải loại bỏ chúng
3. Cột p1_point_won, p2_point_won chứa dữ liệu kiểu string, chúng tôi chuẩn hóa về dạng số( AD thành 50 điểm)
4. Dùng biểu đồ box_plot để phát hiện các khoảng chứ outlier và lọc outlier.




In [ ]:
# 1. Đọc file CSV
# Đọc dữ liệu từ file CSV gốc
file_path = "/content/Wimbledon_featured_matches .csv"
df = pd.read_csv(file_path)

# 2. Xóa các cột không cần thiết
# Xóa các cột không được sử dụng trong phân tích
columns_to_drop = ['serve_width', 'serve_depth', 'return_depth']
df.drop(columns=columns_to_drop, inplace=True)

# 3. Loại bỏ các hàng có giá trị NA trong cột speed_mph
# Xóa các hàng chứa giá trị bị thiếu (NA) trong cột speed_mph
df = df.dropna(subset=['speed_mph'])

# Áp dụng chuẩn hóa cho các cột p1_points_won và p2_points_won
df['p1_points_won'] = df['p1_points_won'].apply(normalize_points)
df['p2_points_won'] = df['p2_points_won'].apply(normalize_points)

# Loại bỏ các hàng không hợp lệ sau khi chuẩn hóa
df = df.dropna(subset=['p1_points_won', 'p2_points_won'])

# 5. Vẽ biểu đồ boxplot để phát hiện và lọc outlier
# Danh sách các cột cần kiểm tra outlier
columns_to_analyze = ["p1_distance_run", "p2_distance_run", "rally_count", "speed_mph"]

# Tạo thư mục để lưu các hình ảnh boxplot
output_dir = "/content/boxplot_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Vẽ boxplot và lưu hình ảnh
for var in columns_to_analyze:
    sns.boxplot(x=df[var], color="darkorange")  # Màu cam cho boxplot
    plt.title(f"Boxplot of {var}", fontsize=14)  # Tiêu đề boxplot
    plt.xlabel(var, fontsize=12)  # Nhãn trục x
    plt.grid(axis="x", linestyle="--", alpha=0.6)  # Thêm lưới để dễ quan sát
    plt.savefig(f"{output_dir}/{var}_boxplot.png")  # Lưu hình ảnh vào thư mục
    plt.close()

print(f"Biểu đồ boxplot đã được lưu vào thư mục: {output_dir}")

# 6. Lọc các giá trị ngoại lai (outlier)
# Giữ lại các hàng với điều kiện lọc outlier cho từng cột
df = df[
    (df['p1_distance_run'] <= 100) &
    (df['p2_distance_run'] <= 100) &
    (df['rally_count'] <= 20) &
    (df['speed_mph'] >= 80)
]

# Tạo thư mục để lưu hình ảnh boxplot sau khi lọc
output_dir1 = "/content/boxplot_images1"
if not os.path.exists(output_dir1):
    os.makedirs(output_dir1)

# Vẽ lại boxplot và lưu hình ảnh sau khi lọc
for var in columns_to_analyze:
    sns.boxplot(x=df[var], color="darkorange")  # Màu cam cho boxplot
    plt.title(f"Boxplot of {var} (After Filtering)", fontsize=14)  # Tiêu đề boxplot
    plt.xlabel(var, fontsize=12)  # Nhãn trục x
    plt.grid(axis="x", linestyle="--", alpha=0.6)  # Thêm lưới để dễ quan sát
    plt.savefig(f"{output_dir1}/{var}_boxplot.png")  # Lưu hình ảnh vào thư mục
    plt.close()

Biểu đồ boxplot đã được lưu vào thư mục: /content/boxplot_images


Standardization of data by min_max_scaler

In [ ]:
columns_to_scale = ['p1_distance_run', 'p2_distance_run', 'rally_count',
                    'speed_mph', 'p1_points_won', 'p2_points_won']
scaler = MinMaxScaler()

# Chuẩn hóa các cột
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
# Lưu file CSV sau khi xử lý
# Ghi dữ liệu đã xử lý vào file CSV mới
output_file_path = "/content/Wimbledon_featured_matches_cleaned.csv"
df.to_csv(output_file_path, index=False)